# Pushup Form Checker (Final Fixed Version)

This notebook implements a hybrid approach to Pushup Form checking:
1. **Geometric Analysis:** Uses specific angles (elbows, hips) to judge form (The "White Box" approach).
2. **AI Classifier:** Uses a lightweight YOLOv8-Classifier trained on your dataset to verify the result.

### Fixes Implemented:
* **Model Mismatch Fixed:** Now uses `yolov8n-cls.pt` for classification training instead of `pose`.
* **Overfitting Fixed:** Epochs reduced to 15, Patience set to 5.
* **Visual Feedback:** Draws the skeleton and changes color based on form.

In [21]:
# 1. Installs & Imports
%pip install ultralytics opencv-python numpy pandas matplotlib
%pip install scikit-learn joblib

import cv2, os, math, time, random
import numpy as np
import pandas as pd
import torch # Import PyTorch to check for CUDA
from pathlib import Path
from collections import deque
from ultralytics import YOLO

# --- GPU/CPU Device Selection ---
# This logic mirrors your existing project. It checks for a CUDA-compatible GPU.
# If CUDA is available, it uses device index 0 (the first GPU); otherwise, it uses 'cpu'.
DEVICE = 0 if torch.cuda.is_available() else 'cpu'
print(f"Device selected for Training & Inference: {DEVICE}")
print("-" * 40)


# --- CONFIGURATION ---
# UPDATE THIS PATH TO YOUR ACTUAL DATA LOCATION IF NECESSARY
DATASET_DIR = "C:/Data/hboict/Sem7-AIFS/Personal_Project" 
OUTPUT_FRAMES_DIR = "data/frames_v2"
RUNS_DIR = "runs"
RUN_BASE_NAME = "pushup-cls-v2" # YOLO will auto-increment this (e.g., v22, v23)

# Hyperparameters (Optimized to prevent overfitting)
TRAINING_EPOCHS = 4
TRAINING_PATIENCE = 1 

# Define Keypoint Indices for YOLOv8-Pose
KS = {
    "nose": 0, "ls": 5, "rs": 6, "le": 7, "re": 8, "lw": 9, "rw": 10, 
    "lh": 11, "rh": 12, "lk": 13, "rk": 14, "la": 15, "ra": 16
}

print("Configuration set. Run next cell to start training.")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Device selected for Training & Inference: cpu
----------------------------------------
Configuration set. Run next cell to start training.


### 1.5 Gathering new frames

In [7]:
import cv2
import os
import random
from pathlib import Path
import shutil

# --- CONFIGURATION ---
# Base directory where your 'Correct Sequence' and 'Wrong Sequence' video folders live
DATASET_DIR = Path("C:/Data/hboict/Sem7-AIFS/Personal_Project") 

# Output directory for the new, large dataset (Make sure this is set in Cell 1 config later!)
OUTPUT_FRAMES_V2_DIR = Path("data/frames_v2") 

# Adjust this number to control the data multiplier:
# 1 = extract every frame (massive data)
# 5 = extract every 5th frame (recommended multiplier ~5-10x)
FRAME_SKIP = 5 
# ---------------------

def extract_and_split_data(video_folder_name, output_base_dir):
    """Extracts frames from all videos in a source folder, saving them to new T/V/T splits."""
    source_dir = DATASET_DIR / video_folder_name
    
    # Create all output directories upfront: train/correct, val/correct, etc.
    splits = ["train", "val", "test"]
    for split in splits:
        (output_base_dir / split / video_folder_name).mkdir(parents=True, exist_ok=True)

    video_files = list(source_dir.glob("*.mp4"))
    
    # --- Split Video Files (70% train, 15% val, 15% test) ---
    random.shuffle(video_files)
    total_videos = len(video_files)
    train_split_count = int(total_videos * 0.7)
    val_split_count = int(total_videos * 0.15)
    
    train_files = video_files[:train_split_count]
    val_files = video_files[train_split_count:train_split_count + val_split_count]
    test_files = video_files[train_split_count + val_split_count:]
    
    file_map = {"train": train_files, "val": val_files, "test": test_files}

    total_frames_extracted = 0
    
    for split, files in file_map.items():
        for video_path in files:
            cap = cv2.VideoCapture(str(video_path))
            frame_count = 0
            frames_saved_from_video = 0
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Sample every FRAME_SKIP frame
                if frame_count % FRAME_SKIP == 0:
                    # Naming convention includes video name and frame number
                    output_file_name = f"{video_path.stem}_f{frame_count}.jpg"
                    output_path = output_base_dir / split / video_folder_name / output_file_name
                    cv2.imwrite(str(output_path), frame)
                    frames_saved_from_video += 1
                
                frame_count += 1
            
            cap.release()
            total_frames_extracted += frames_saved_from_video
            
    print(f"Extracted {total_frames_extracted} frames for '{video_folder_name}' and split into T/V/T.")
    return total_frames_extracted

def run_extraction_pipeline():
    # 1. Clean up old data structure if desired (optional)
    if OUTPUT_FRAMES_V2_DIR.exists():
        print(f"Clearing old directory: {OUTPUT_FRAMES_V2_DIR}")
        shutil.rmtree(OUTPUT_FRAMES_V2_DIR)
        
    # 2. Run extraction for both classes
    total_correct = extract_and_split_data("Correct Sequence", OUTPUT_FRAMES_V2_DIR)
    total_incorrect = extract_and_split_data("Wrong Sequence", OUTPUT_FRAMES_V2_DIR)
    
    print("\n--- Summary ---")
    print(f"✅ Extraction Complete! Total Frames: {total_correct + total_incorrect}")
    print(f"New data ready in: {OUTPUT_FRAMES_V2_DIR}")
    print(f"Old data folder was: data/frames. New folder is: data/frames_v2")

run_extraction_pipeline()

Extracted 921 frames for 'Correct Sequence' and split into T/V/T.
Extracted 1304 frames for 'Wrong Sequence' and split into T/V/T.

--- Summary ---
✅ Extraction Complete! Total Frames: 2225
New data ready in: data\frames_v2
Old data folder was: data/frames. New folder is: data/frames_v2


## 1.75?
SEt images to csv

In [6]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import random
from math import sqrt
from ultralytics import YOLO # <-- REQUIRED IMPORT for Keypoint Detection

# --- CONFIGURATION ---
# The input is the direct output of Cell 1.5 (the image frames)
FRAME_INPUT_DIR = Path('data/frames_v2') 
OUTPUT_BASE_DIR = Path('.') # Save the final CSVs in the root directory

# Initialize Pose Model
POSE_MODEL_PATH = "yolov8n-pose.pt"
try:
    pose_model = YOLO(POSE_MODEL_PATH) 
    print(f"YOLOv8 Pose Model loaded successfully from '{POSE_MODEL_PATH}'.")
except Exception as e:
    print(f"Error loading YOLO model: {e}. Please ensure '{POSE_MODEL_PATH}' is available.")
    pose_model = None

# --- Keypoint Structure ---
# This dictionary maps joint names to the index used by the 17-point COCO model
KS = {
    'nose': 0, 'le': 3, 're': 4, 'ls': 5, 'rs': 6, 'le': 7, 're': 8,
    'lw': 9, 'rw': 10, 'lh': 11, 'rh': 12, 'lk': 13, 'rk': 14,
    'la': 15, 'ra': 16
}

# --- Helper Functions (Robust Features) ---

def calculate_angle(kps, p1, p2, p3):
    """Calculates the angle (in degrees) between three keypoints, centered at p2."""
    p1_coords = kps[p1][:2]
    p2_coords = kps[p2][:2]
    p3_coords = kps[p3][:2]
    
    v1 = np.array(p1_coords) - np.array(p2_coords)
    v2 = np.array(p3_coords) - np.array(p2_coords)
    
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    if norm_v1 == 0 or norm_v2 == 0:
        return 180.0
        
    angle_rad = np.arccos(np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0))
    return np.degrees(angle_rad)

def create_robust_features(kps):
    """Creates a 20-feature vector using angles and distance ratios (Invariant to rotation/scale)."""
    features = []

    def dist(p1, p2):
        return np.linalg.norm(kps[p1, :2] - kps[p2, :2])

    # 1. 8 Angles (Joint Flexure)
    features.append(calculate_angle(kps, KS['rw'], KS['re'], KS['rs']))
    features.append(calculate_angle(kps, KS['lw'], KS['le'], KS['ls']))
    features.append(calculate_angle(kps, KS['re'], KS['rs'], KS['rh']))
    features.append(calculate_angle(kps, KS['le'], KS['ls'], KS['lh']))
    features.append(calculate_angle(kps, KS['rs'], KS['rh'], KS['rk']))
    features.append(calculate_angle(kps, KS['ls'], KS['lh'], KS['lk']))
    features.append(calculate_angle(kps, KS['rh'], KS['rk'], KS['ra']))
    features.append(calculate_angle(kps, KS['lh'], KS['lk'], KS['la']))

    # 2. 12 Distance Ratios
    torso_length = (dist(KS['ls'], KS['lh']) + dist(KS['rs'], KS['rh'])) / 2
    if torso_length == 0: torso_length = 1.0

    features.append(dist(KS['rs'], KS['re']) / torso_length) 
    features.append(dist(KS['re'], KS['rw']) / torso_length) 
    features.append(dist(KS['ls'], KS['le']) / torso_length) 
    features.append(dist(KS['le'], KS['lw']) / torso_length) 
    features.append(dist(KS['rs'], KS['ls']) / torso_length) 
    features.append(dist(KS['rh'], KS['lh']) / torso_length) 
    features.append(dist(KS['rh'], KS['rk']) / torso_length) 
    features.append(dist(KS['rk'], KS['ra']) / torso_length) 
    features.append(dist(KS['lh'], KS['lk']) / torso_length) 
    features.append(dist(KS['lk'], KS['la']) / torso_length) 
    features.append(dist(KS['nose'], KS['rs']) / torso_length) 
    features.append(dist(KS['nose'], KS['ls']) / torso_length) 
    
    return features

def augment_and_extract(kps, num_augmentations=5):
    """Applies small random noise/scaling to keypoints and extracts features multiple times."""
    
    all_features = []
    
    # 1. Base features (non-augmented)
    all_features.append(create_robust_features(kps.copy()))
    
    # 2. Augmentations (4 more times)
    for _ in range(num_augmentations - 1):
        kps_aug = kps.copy()
        
        # Random Scaling: +/- 5% 
        scale_factor = 1 + random.uniform(-0.05, 0.05) 
        kps_aug[:, :2] *= scale_factor
        
        # Random Noise: +/- 0.002 (Simulates minor detection flicker)
        noise = np.random.uniform(-0.002, 0.002, size=kps_aug[:, :2].shape)
        kps_aug[:, :2] += noise
        
        all_features.append(create_robust_features(kps_aug))

    return all_features


# --- Main Data Creation Loop ---

if pose_model:
    splits = ["train", "val", "test"]
    feature_names = (
        [f'angle_{i}' for i in range(8)] + 
        [f'ratio_{i}' for i in range(12)] + 
        ['label']
    )
        
    total_samples = 0
    for split in splits:
        all_data = []
        split_dir = FRAME_INPUT_DIR / split
        
        if not split_dir.exists():
            print(f"⚠️ Skipping {split}: Frame directory not found at {split_dir}.")
            continue
            
        # Search recursively for image files (.jpg)
        image_files = list(split_dir.glob("*/*.jpg"))
        
        if not image_files:
            print(f"⚠️ No image files found in {split_dir}.")
            continue
            
        print(f"Processing {len(image_files)} images for {split.upper()} split...")

        for filepath in image_files:
            # 1. Get Label
            label_raw = filepath.parent.name
            label = None
            if 'correct' in label_raw.lower():
                label = 'correct'
            elif 'wrong' in label_raw.lower():
                label = 'incorrect'
            else:
                continue
            
            try:
                # 2. Keypoint Detection (Inference)
                # Using the string representation of Path object for YOLO input
                results = pose_model(str(filepath), verbose=False)
                
                # Check if a person was detected
                if results and results[0].keypoints.xyn.shape[0] > 0:
                    
                    # Extract normalized coordinates (xyn) and confidence
                    xy_normalized = results[0].keypoints.xyn[0].cpu().numpy()
                    conf_values = results[0].keypoints.conf[0].cpu().numpy().reshape(-1, 1)
                    # Keypoints in [x, y, confidence] format (normalized to 0-1)
                    kps_normalized_xyc = np.concatenate((xy_normalized, conf_values), axis=1)
                    
                    if kps_normalized_xyc.shape == (17, 3):
                        # 3. Augment and Extract Features
                        robust_feature_vectors = augment_and_extract(kps_normalized_xyc, num_augmentations=5)
                        
                        for features in robust_feature_vectors:
                            features.append(label)
                            all_data.append(features)
                        
            except Exception as e:
                # print(f"Skipping file {filepath.name} due to error: {e}")
                continue

        # 4. Save Final DataFrame for the split
        df_robust = pd.DataFrame(all_data, columns=feature_names)
        output_filename = f'pushup_dataset_robust_{split}.csv'
        df_robust.to_csv(OUTPUT_BASE_DIR / output_filename, index=False)
        
        total_samples += len(df_robust)
        print(f"✅ {split.upper()} Data Generated: {len(df_robust)} robust samples. Saved to {output_filename}")
        print(f"   Class Balance: {df_robust['label'].value_counts().to_dict()}")

    print(f"\n--- Total Robust Samples Generated: {total_samples} ---")
else:
    print("Cannot run data creation: YOLO model failed to load.")

YOLOv8 Pose Model loaded successfully from 'yolov8n-pose.pt'.
Processing 1543 images for TRAIN split...
✅ TRAIN Data Generated: 7715 robust samples. Saved to pushup_dataset_robust_train.csv
   Class Balance: {'incorrect': 4465, 'correct': 3250}
Processing 376 images for VAL split...
✅ VAL Data Generated: 1880 robust samples. Saved to pushup_dataset_robust_val.csv
   Class Balance: {'incorrect': 1190, 'correct': 690}
Processing 306 images for TEST split...
✅ TEST Data Generated: 1530 robust samples. Saved to pushup_dataset_robust_test.csv
   Class Balance: {'incorrect': 865, 'correct': 665}

--- Total Robust Samples Generated: 11125 ---


### 2. Train the Classifier (Correctly)
We use `yolov8n-cls.pt` (Classification model) and stricter settings to prevent overfitting.

In [7]:
# def train_classifier():
#     # Load the CLASSIFICATION model (yolov8n-cls.pt)
#     model = YOLO('yolov8n-cls.pt')  

#     print(f"Starting training on device: {DEVICE}")
#     results = model.train(
#         data=OUTPUT_FRAMES_DIR,  
#         epochs=TRAINING_EPOCHS,    
#         patience=TRAINING_PATIENCE, 
#         imgsz=224,               
#         project=RUNS_DIR,        
#         name=RUN_BASE_NAME,      
#         batch=64,                
#         augment=True,            
#         device=DEVICE,           # <-- FORCES GPU USAGE
#         exist_ok=False      # for automatic run counting
#     )
#     return results

# # Run training
# train_classifier()
# print("\nTraining Complete. Best weights saved.")

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import joblib

def train_pose_classifier():
    # --- CONFIGURATION ---
    # ⚠️ IMPORTANT: Load the robust, augmented training data.
    CSV_PATH = 'pushup_dataset_robust_train.csv' 
    MODEL_SAVE_PATH = 'pose_classifier_robust.pkl'
    
    # 1. Load Data
    try:
        df = pd.read_csv(CSV_PATH)
    except FileNotFoundError:
        print(f"ERROR: CSV file not found at {CSV_PATH}. Run the corrected Cell 1.75 first.")
        return
        
    X = df.drop("label", axis=1)
    y = df["label"]
    
    # 2. Split (using a small validation set from the already prepared training data)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.15, # Use a smaller test set here (15%)
        random_state=42,
        stratify=y       # Ensure balanced classes in the split
    )
    
    # 3. Train Random Forest (The "Brain")
    # Added hyperparameter optimization: max_depth to prevent minor overfitting 
    # to noisy augmented features.
    clf = RandomForestClassifier(
        n_estimators=150,       # Slight increase in trees for better generalization
        max_depth=15,           # Limits tree depth to prevent deep overfitting
        random_state=42,
        class_weight='balanced' # Highly recommended due to potential data imbalance
    )
    clf.fit(X_train, y_train)
    
    # 4. Evaluate
    y_pred = clf.predict(X_test)
    print("--- Model Evaluation on Internal Test Split ---")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    
    # 5. Save Model
    joblib.dump(clf, MODEL_SAVE_PATH)
    print(f"✅ Model saved as '{MODEL_SAVE_PATH}'")

train_pose_classifier()

--- Model Evaluation on Internal Test Split ---
Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

     correct       1.00      1.00      1.00       488
   incorrect       1.00      1.00      1.00       670

    accuracy                           1.00      1158
   macro avg       1.00      1.00      1.00      1158
weighted avg       1.00      1.00      1.00      1158

✅ Model saved as 'pose_classifier_robust.pkl'


## Hyperparameter tuning

In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, recall_score, classification_report
import joblib

# --- Load TRAIN and VAL for tuning ---
df_train = pd.read_csv("pushup_dataset_robust_train.csv")
df_val   = pd.read_csv("pushup_dataset_robust_val.csv")

X_train = df_train.drop("label", axis=1)
y_train = df_train["label"]

X_val = df_val.drop("label", axis=1)
y_val = df_val["label"]

# --- Custom scorer: emphasize recall on the 'correct' class ---
def recall_correct(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label='correct')

recall_correct_scorer = make_scorer(recall_correct)

# --- Base RF ---
rf_base = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

# --- Hyperparameter search space ---
param_dist = {
    "n_estimators":   [80, 120, 160, 220],
    "max_depth":      [6, 8, 10, 12],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf":  [1, 2, 4],
    "max_features":   ["sqrt", "log2", 0.5]
}

search = RandomizedSearchCV(
    rf_base,
    param_distributions=param_dist,
    n_iter=20,             # you can bump this up if it runs fast
    scoring=recall_correct_scorer,  # prioritise catching 'correct' reps
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

print("Best params:", search.best_params_)
print("Best CV recall(correct):", search.best_score_)

# Evaluate the tuned model on VAL set
best_rf = search.best_estimator_
y_val_pred = best_rf.predict(X_val)
print("\n--- VAL SET PERFORMANCE (tuned RF) ---")
print(classification_report(y_val, y_val_pred))

# Save tuned model for later test + live loop
joblib.dump(best_rf, "pose_classifier_robust_tuned.pkl")
print("\nSaved tuned RF model as 'pose_classifier_robust_tuned.pkl'")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params: {'n_estimators': 160, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 6}
Best CV recall(correct): 0.5584615384615385

--- VAL SET PERFORMANCE (tuned RF) ---
              precision    recall  f1-score   support

     correct       0.51      0.82      0.63       690
   incorrect       0.84      0.55      0.67      1190

    accuracy                           0.65      1880
   macro avg       0.68      0.69      0.65      1880
weighted avg       0.72      0.65      0.65      1880


Saved tuned RF model as 'pose_classifier_robust_tuned.pkl'


## number whatever, visualisations

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import numpy as np
import os
import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import learning_curve  # <-- NEW

# --- CONFIGURATION ---
TRAIN_CSV_PATH = "pushup_dataset_robust_train.csv"
TEST_CSV_PATH  = "pushup_dataset_robust_test.csv"
MODEL_PATH     = "pose_classifier_robust.pkl"

# --- FOLDER SETUP ---
BASE_VIS_DIR = "rf_visuals"
RUN_ID = time.strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = os.path.join(BASE_VIS_DIR, RUN_ID)

# --- FEATURE DEFINITION MAP ---
FEATURE_MAP = {
    'angle_0': 'R Elbow Angle', 'angle_1': 'L Elbow Angle',
    'angle_2': 'R Shoulder Angle', 'angle_3': 'L Shoulder Angle',
    'angle_4': 'R Hip (Torso) Angle', 'angle_5': 'L Hip (Torso) Angle',
    'angle_6': 'R Knee Angle', 'angle_7': 'L Knee Angle',
    'ratio_0': 'R Shoulder-Elbow / Torso', 'ratio_1': 'R Elbow-Wrist / Torso',
    'ratio_2': 'L Shoulder-Elbow / Torso', 'ratio_3': 'L Elbow-Wrist / Torso',
    'ratio_4': 'Shoulder Width / Torso', 'ratio_5': 'Hip Width / Torso',
    'ratio_6': 'R Hip-Knee / Torso', 'ratio_7': 'R Knee-Ankle / Torso',
    'ratio_8': 'L Hip-Knee / Torso', 'ratio_9': 'L Knee-Ankle / Torso',
    'ratio_10': 'Nose-R Shoulder / Torso', 'ratio_11': 'Nose-L Shoulder / Torso',
}

try:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Saving all visualizations and reports to folder: {OUTPUT_DIR}\n")

    # 1. Load data + model
    df_train = pd.read_csv(TRAIN_CSV_PATH)
    df_test  = pd.read_csv(TEST_CSV_PATH)
    rf_model = joblib.load(MODEL_PATH)

    # 2. Prepare TEST data (NO re-splitting)
    X_test = df_test.drop("label", axis=1)
    y_test = df_test["label"]
    y_pred = rf_model.predict(X_test)

    # --- VIS 1: Class distribution (test set) ---
    class_counts = y_test.value_counts().sort_index()
    plt.figure(figsize=(7, 5))
    bars = plt.bar(class_counts.index, class_counts.values,
                   color=['skyblue', 'lightcoral'])
    plt.title('1. Class Distribution (Test Set): Correct vs. Incorrect Poses')
    plt.xlabel('Pose Class')
    plt.ylabel('Number of Samples')
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, h + 5, str(int(h)),
                 ha='center', va='bottom')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'class_distribution.png'))
    plt.close()
    print("Plot 1/5: Class distribution saved.")

    # --- VIS 2: Feature importance ---
    feature_names = [f'angle_{i}' for i in range(8)] + [f'ratio_{i}' for i in range(12)]
    importances = rf_model.feature_importances_

    fi_df = (pd.DataFrame({'Feature': feature_names,
                           'Importance': importances})
               .sort_values(by='Importance', ascending=False))
    fi_df['Description'] = fi_df['Feature'].map(FEATURE_MAP)

    plt.figure(figsize=(12, 8))
    top_features = fi_df.head(15)
    plt.barh(top_features['Description'], top_features['Importance'], color='purple')
    plt.gca().invert_yaxis()
    plt.title('2. Random Forest Feature Importance (Robust Features)')
    plt.xlabel('Importance Score')
    plt.ylabel('Feature Description')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'feature_importance.png'))
    plt.close()
    print("Plot 2/5: Feature importance saved.")

    # --- VIS 3: Confusion matrix (raw counts) ---
    labels = sorted(df_test['label'].unique())
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

    plt.figure(figsize=(6, 6))
    disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
    plt.title('3. Confusion Matrix (Raw Counts)')
    plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix_raw.png'))
    plt.close()
    print("Plot 3/5: Confusion Matrix (Raw) saved.")

    # --- VIS 4: Confusion matrix (normalized) ---
    cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    disp_norm = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=labels)

    plt.figure(figsize=(6, 6))
    disp_norm.plot(cmap=plt.cm.Greens, values_format='.2f', ax=plt.gca())
    plt.title('4. Confusion Matrix (Normalized Percentages)')
    plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix_normalized.png'))
    plt.close()
    print("Plot 4/5: Confusion Matrix (Normalized) saved.")

    # --- Classification report (same as you printed) ---
    report = classification_report(y_test, y_pred)
    report_path = os.path.join(OUTPUT_DIR, 'classification_report.txt')
    with open(report_path, 'w') as f:
        f.write(report)

    print("\n--- CLASSIFICATION REPORT (Test Set) ---")
    print(report)
    print(f"Numerical report saved to '{report_path}'.")

    # --- VIS 5: "results.png"-style learning-curve plot ---------------------

    # Use training data for learning curve
    X_train_lc = df_train.drop("label", axis=1)
    y_train_lc = df_train["label"]

    train_sizes, train_scores, val_scores = learning_curve(
        rf_model,
        X_train_lc,
        y_train_lc,
        cv=5,
        scoring="accuracy",
        train_sizes=np.linspace(0.1, 1.0, 10),
        shuffle=True,
        random_state=42,
        n_jobs=-1
    )

    train_mean = train_scores.mean(axis=1)
    val_mean   = val_scores.mean(axis=1)

    # Treat "loss" as 1 - accuracy (just for plotting)
    loss_train = 1.0 - train_mean
    loss_val   = 1.0 - val_mean

    def smooth_curve(y, window=3):
        """Simple moving-average smoothing."""
        if len(y) <= window:
            return y
        kernel = np.ones(window) / window
        return np.convolve(y, kernel, mode="same")

    train_mean_s = smooth_curve(train_mean)
    val_mean_s   = smooth_curve(val_mean)
    loss_train_s = smooth_curve(loss_train)
    loss_val_s   = smooth_curve(loss_val)

    fig, axes = plt.subplots(2, 2, figsize=(8, 8))

    # top-left: train loss
    axes[0, 0].plot(train_sizes, loss_train, marker="o", label="results")
    axes[0, 0].plot(train_sizes, loss_train_s, linestyle="dotted", label="smooth")
    axes[0, 0].set_title("train/loss")
    axes[0, 0].set_xlabel("Training samples")
    axes[0, 0].set_ylabel("Loss (1 - accuracy)")

    # top-right: train accuracy
    axes[0, 1].plot(train_sizes, train_mean, marker="o", label="results")
    axes[0, 1].plot(train_sizes, train_mean_s, linestyle="dotted", label="smooth")
    axes[0, 1].set_title("metrics/accuracy_train")
    axes[0, 1].set_xlabel("Training samples")
    axes[0, 1].set_ylabel("Accuracy")
    axes[0, 1].legend()

    # bottom-left: val loss
    axes[1, 0].plot(train_sizes, loss_val, marker="o", label="results")
    axes[1, 0].plot(train_sizes, loss_val_s, linestyle="dotted", label="smooth")
    axes[1, 0].set_title("val/loss")
    axes[1, 0].set_xlabel("Training samples")
    axes[1, 0].set_ylabel("Loss (1 - accuracy)")

    # bottom-right: val accuracy
    axes[1, 1].plot(train_sizes, val_mean, marker="o", label="results")
    axes[1, 1].plot(train_sizes, val_mean_s, linestyle="dotted", label="smooth")
    axes[1, 1].set_title("metrics/accuracy_val")
    axes[1, 1].set_xlabel("Training samples")
    axes[1, 1].set_ylabel("Accuracy")

    for ax in axes.flat:
        ax.grid(True)

    fig.tight_layout()
    curve_path = os.path.join(OUTPUT_DIR, "learning_curves.png")
    fig.savefig(curve_path)
    plt.close(fig)
    print("Plot 5/5: Learning-curve style metrics saved.")

except FileNotFoundError as e:
    print("\n--- ERROR ---")
    print(f"Required file not found: {e.filename}. Make sure the CSVs and model exist before running this cell.")
except Exception as e:
    print("\n--- FATAL ERROR DURING VISUALIZATION ---")
    print(f"An unexpected error occurred: {e}")


Saving all visualizations and reports to folder: rf_visuals\20251207_154719

Plot 1/5: Class distribution saved.
Plot 2/5: Feature importance saved.
Plot 3/5: Confusion Matrix (Raw) saved.
Plot 4/5: Confusion Matrix (Normalized) saved.

--- CLASSIFICATION REPORT (Test Set) ---
              precision    recall  f1-score   support

     correct       0.89      0.81      0.85       665
   incorrect       0.86      0.92      0.89       865

    accuracy                           0.87      1530
   macro avg       0.88      0.87      0.87      1530
weighted avg       0.88      0.87      0.87      1530

Numerical report saved to 'rf_visuals\20251207_154719\classification_report.txt'.
Plot 5/5: Learning-curve style metrics saved.


## test

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Train on the dedicated TRAIN CSV
df_train = pd.read_csv("pushup_dataset_robust_train.csv")
X_train = df_train.drop("label", axis=1)
y_train = df_train["label"]

clf = RandomForestClassifier(
    n_estimators=150,
    max_depth=15,
    random_state=42,
    class_weight='balanced'
)
clf.fit(X_train, y_train)

# Test on the completely separate TEST CSV
df_test = pd.read_csv("pushup_dataset_robust_test.csv")
X_test = df_test.drop("label", axis=1)
y_test = df_test["label"]

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# (optional) overwrite the model used by the live loop
import joblib
joblib.dump(clf, "pose_classifier_robust.pkl")


              precision    recall  f1-score   support

     correct       0.89      0.81      0.85       665
   incorrect       0.86      0.92      0.89       865

    accuracy                           0.87      1530
   macro avg       0.88      0.87      0.87      1530
weighted avg       0.88      0.87      0.87      1530



['pose_classifier_robust.pkl']

### 3. Geometric Logic Helper Functions
These functions calculate angles to check for depth and hip sag.

In [26]:
# import numpy as np

# def calculate_angle(a, b, c):
#     """Calculates angle ABC (in degrees) where B is the vertex."""
#     a, b, c = np.array(a), np.array(b), np.array(c)
#     ba = a - b
#     bc = c - b
    
#     norm_ba = np.linalg.norm(ba)
#     norm_bc = np.linalg.norm(bc)
#     if norm_ba == 0 or norm_bc == 0:
#         return 180.0
        
#     cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
#     angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
#     return np.degrees(angle)

# def get_xy(kps, index):
#     """Safe helper to get (x, y) tuple from keypoints."""
#     return (float(kps[index, 0]), float(kps[index, 1]))

# def get_best_elbow_angle(kps):
#     """
#     Returns the best elbow angle (left or right) based on confidence.
#     Falls back to None if nothing reliable is found.
#     """
#     # Assuming KS (Keypoint Skeleton) is defined in your environment
#     if 'ls' not in KS or 'le' not in KS or 'lw' not in KS:
#         return None
        
#     left_conf = kps[KS['ls'], 2] * kps[KS['le'], 2] * kps[KS['lw'], 2]
#     right_conf = kps[KS['rs'], 2] * kps[KS['re'], 2] * kps[KS['rw'], 2]
    
#     elbow_angle = None
    
#     if left_conf > right_conf and left_conf > 0.5:
#         elbow_angle = calculate_angle(
#             get_xy(kps, KS['ls']),
#             get_xy(kps, KS['le']),
#             get_xy(kps, KS['lw'])
#         )
#     elif right_conf > 0.5:
#         elbow_angle = calculate_angle(
#             get_xy(kps, KS['rs']),
#             get_xy(kps, KS['re']),
#             get_xy(kps, KS['rw'])
#         )
    
#     return elbow_angle

# def assess_form_geometric(kps, phase="top", bottom_elbow_angle=None):
#     """
#     Returns a list of issues based on geometry (angles / alignment).
#     Lockout check removed for stability.
#     """
#     issues = []
    
#     # --- 1. ELBOW ANGLES: DEPTH (BOTTOM) ---
    
#     # Bottom depth check (using min elbow angle from the down phase)
#     if bottom_elbow_angle is not None:
#         # Relaxed threshold: Must be 110 degrees or less
#         if bottom_elbow_angle > 110:
#             issues.append(f"Go deeper! (Min elbow angle {int(bottom_elbow_angle)}° > 110°)")
    
#     # NOTE: Top lockout check removed due to instability from keypoint jitter.

#     # --- 2. BODY LINE + HIP SAG / PIKE (Shoulder–Hip–Knee/Ankle) ---
#     def check_body_side(side_prefixes):
#         """Returns the most critical body line issue, or None."""
#         s_key, h_key, k_key, a_key = side_prefixes
        
#         # Check if all required keys exist in KS
#         if not all(key in KS for key in side_prefixes): return None
        
#         s_idx, h_idx, k_idx, a_idx = KS[s_key], KS[h_key], KS[k_key], KS[a_key]
#         conf = kps[s_idx, 2] * kps[h_idx, 2] * kps[k_idx, 2]
#         if conf <= 0.5: return None
        
#         shoulder = get_xy(kps, s_idx)
#         hip      = get_xy(kps, h_idx)
#         knee     = get_xy(kps, k_idx)
#         ankle    = get_xy(kps, a_idx)
        
#         # Primary Check: S-H-A Angle
#         body_angle = calculate_angle(shoulder, hip, ankle)
#         if body_angle < 165:
#             return f"Keep body straighter (S-H-A angle {int(body_angle)}° < 165°)"

#         # Secondary Check: Sag/Pike offset (only if S-H-A angle passes)
#         sh = np.array(shoulder)
#         hp = np.array(hip)
#         kn = np.array(knee)
#         seg_len = np.linalg.norm(kn - sh)
        
#         if seg_len > 0:
#             t = np.linalg.norm(hp - sh) / seg_len
#             expected_hip_y = sh[1] + (kn[1] - sh[1]) * t
#             hip_offset = hp[1] - expected_hip_y
#             sag_thresh = 0.1 * seg_len # 10% of body segment as threshold
            
#             if hip_offset > sag_thresh:
#                 return "Hips sagging – tighten your core."
#             elif hip_offset < -sag_thresh:
#                 return "Hips too high – avoid piking."
                
#         return None

#     # Try left side first, then right
#     body_issue = check_body_side(('ls', 'lh', 'lk', 'la'))
#     if body_issue is None:
#         body_issue = check_body_side(('rs', 'rh', 'rk', 'ra'))
    
#     if body_issue:
#         issues.append(body_issue)
    
#     # Fallback to original simple check if full body points aren't available
#     if not body_issue and kps[KS['ls'], 2] > 0.5 and kps[KS['lh'], 2] > 0.5 and kps[KS['lk'], 2] > 0.5:
#          body_angle = calculate_angle(get_xy(kps, KS['ls']), get_xy(kps, KS['lh']), get_xy(kps, KS['lk']))
#          if body_angle < 165:
#               issues.append(f"Fix hips! (Body angle {int(body_angle)}°, must be > 165°)")
            
#     return issues

import numpy as np
# Assuming KS is defined elsewhere

def calculate_angle(a, b, c):
    """Calculates angle ABC (in degrees) where B is the vertex."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    
    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0:
        return 180.0
        
    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def get_xy(kps, index):
    """Safe helper to get (x, y) tuple from keypoints."""
    return (float(kps[index, 0]), float(kps[index, 1]))

def get_best_elbow_angle(kps):
    """
    Returns the best elbow angle (left or right) based on confidence.
    (Needed for min_elbow_angle tracking)
    """
    if 'ls' not in KS or 'le' not in KS or 'lw' not in KS:
        return None
        
    left_conf = kps[KS['ls'], 2] * kps[KS['le'], 2] * kps[KS['lw'], 2]
    right_conf = kps[KS['rs'], 2] * kps[KS['re'], 2] * kps[KS['rw'], 2]
    
    elbow_angle = None
    
    if left_conf > right_conf and left_conf > 0.5:
        elbow_angle = calculate_angle(
            get_xy(kps, KS['ls']),
            get_xy(kps, KS['le']),
            get_xy(kps, KS['lw'])
        )
    elif right_conf > 0.5:
        elbow_angle = calculate_angle(
            get_xy(kps, KS['rs']),
            get_xy(kps, KS['le']), # Fixed: was KS['re']
            get_xy(kps, KS['rw'])
        )
    
    return elbow_angle
# REMOVED: assess_form_geometric (The AI Classifier replaces this logic)

### 4. Live Webcam Loop
This block runs the webcam, draws the skeleton, counts reps, and gives feedback.

In [11]:
import cv2
import time
import joblib
import numpy as np
import warnings
from ultralytics import YOLO
from math import sqrt
# from sklearn.exceptions import UserWarning
import os

# --- SUPPRESS WARNINGS ---
warnings.filterwarnings("ignore", category=UserWarning)

# --- 1. CONFIGURATION AND MODEL LOADING ---

# Define Keypoint Structure
KS = {
    'nose': 0, 'le': 3, 're': 4, 'ls': 5, 'rs': 6, 'le': 7, 're': 8,
    'lw': 9, 'rw': 10, 'lh': 11, 'rh': 12, 'lk': 13, 'rk': 14,
    'la': 15, 'ra': 16
}

# Define body segments for drawing the skeleton
SKELETON_LINES = [
    (KS['ls'], KS['rs']), (KS['lh'], KS['rh']), (KS['ls'], KS['lh']), (KS['rs'], KS['rh']), 
    (KS['ls'], KS['le']), (KS['le'], KS['lw']),                                          
    (KS['rs'], KS['re']), (KS['re'], KS['rw']),                                           
    (KS['lh'], KS['lk']), (KS['lk'], KS['la']),                                          
    (KS['rh'], KS['rk']), (KS['rk'], KS['ra'])                                            
]


# Exercise Parameters
ELBOW_ANGLE_THRESHOLD = 125.0    
HIP_ANGLE_THRESHOLD = 160.0     
EXTENDED_ELBOW_THRESHOLD = 165.0 

# >>> ADJUSTED FOR DISTANT WEBCAM START <<<
RELATIVE_MOVE_THRESHOLD = 0.02   # Reduced from 0.05 to 0.02
KP_CONF_MIN = 0.5                # Reduced from 0.8 to 0.5 (Fixes flickering skeleton)
AI_CONF_MIN = 0.55               # Reduced from 0.65 (Optional, for noisier AI input)
# >>> ADJUSTED FOR DISTANT WEBCAM END <<<

POSE_MODEL_PATH = "yolov8n-pose.pt"
CLASSIFIER_MODEL_PATH = "pose_classifier_robust.pkl" 

try:
    pose_model = YOLO(POSE_MODEL_PATH)
    rf_model = joblib.load(CLASSIFIER_MODEL_PATH)
    print("Models loaded successfully. Using Robust Feature Engineering.")
except Exception as e:
    print(f"Error loading models: {e}. Ensure both files are in the directory and the model was re-trained as '{CLASSIFIER_MODEL_PATH}'.")
    pose_model = None
    rf_model = None


# --- 2. HELPER FUNCTIONS (No change needed here) ---

def calculate_angle(kps, p1, p2, p3):
    """Calculates the angle (in degrees) between three keypoints, centered at p2."""
    p1_coords = kps[p1][:2]
    p2_coords = kps[p2][:2]
    p3_coords = kps[p3][:2]
    
    v1 = np.array(p1_coords) - np.array(p2_coords)
    v2 = np.array(p3_coords) - np.array(p2_coords)
    
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    if norm_v1 == 0 or norm_v2 == 0: return 180.0
        
    angle_rad = np.arccos(np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0))
    return np.degrees(angle_rad)

def get_best_elbow_angle(kps):
    """Returns the smallest elbow angle."""
    left_elbow_angle = calculate_angle(kps, KS['lw'], KS['le'], KS['ls'])
    right_elbow_angle = calculate_angle(kps, KS['rw'], KS['re'], KS['rs'])
    return min(left_elbow_angle, right_elbow_angle)

def get_hip_angle(kps):
    """Calculates the angle for hip straightness (Shoulder-Hip-Knee)."""
    left_angle = calculate_angle(kps, KS['ls'], KS['lh'], KS['lk'])
    right_angle = calculate_angle(kps, KS['rs'], KS['rh'], KS['rk'])
    return min(left_angle, right_angle)

def create_robust_features(kps):
    """
    Creates the same 20-feature vector used for training. 
    (8 Angles + 12 Ratios).
    """
    features = []
    def dist(p1, p2): return np.linalg.norm(kps[p1, :2] - kps[p2, :2])

    # 1. 8 Angles
    features.append(calculate_angle(kps, KS['rw'], KS['re'], KS['rs']))
    features.append(calculate_angle(kps, KS['lw'], KS['le'], KS['ls']))
    features.append(calculate_angle(kps, KS['re'], KS['rs'], KS['rh']))
    features.append(calculate_angle(kps, KS['le'], KS['ls'], KS['lh']))
    features.append(calculate_angle(kps, KS['rs'], KS['rh'], KS['rk']))
    features.append(calculate_angle(kps, KS['ls'], KS['lh'], KS['lk']))
    features.append(calculate_angle(kps, KS['rh'], KS['rk'], KS['ra']))
    features.append(calculate_angle(kps, KS['lh'], KS['lk'], KS['la']))

    # 2. 12 Distance Ratios
    torso_length = (dist(KS['ls'], KS['lh']) + dist(KS['rs'], KS['rh'])) / 2
    if torso_length == 0: torso_length = 1.0

    features.append(dist(KS['rs'], KS['re']) / torso_length) 
    features.append(dist(KS['re'], KS['rw']) / torso_length) 
    features.append(dist(KS['ls'], KS['le']) / torso_length) 
    features.append(dist(KS['le'], KS['lw']) / torso_length) 
    features.append(dist(KS['rs'], KS['ls']) / torso_length) 
    features.append(dist(KS['rh'], KS['lh']) / torso_length) 
    features.append(dist(KS['rh'], KS['rk']) / torso_length) 
    features.append(dist(KS['rk'], KS['ra']) / torso_length) 
    features.append(dist(KS['lh'], KS['lk']) / torso_length) 
    features.append(dist(KS['lk'], KS['la']) / torso_length) 
    features.append(dist(KS['nose'], KS['rs']) / torso_length) 
    features.append(dist(KS['nose'], KS['ls']) / torso_length) 
    
    return features


# --- 3. LIVE LOOP STATE MACHINE ---

if pose_model and rf_model:
    cap = cv2.VideoCapture(0)

    # State variables
    state = "up"
    reps = 0
    prev_y = None
    min_elbow_angle = 180.0
    feedback_buffer = None
    default_color = (255, 255, 255) # White

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break

        results = pose_model(frame, verbose=False)
        
        # Keypoint Extraction
        kps = None
        if results and results[0].keypoints.xyn.shape[0] > 0:
            xy_normalized = results[0].keypoints.xyn[0].cpu().numpy()
            conf_values = results[0].keypoints.conf[0].cpu().numpy().reshape(-1, 1)
            kps_normalized_xyc = np.concatenate((xy_normalized, conf_values), axis=1)

            scale_array = np.array([frame.shape[1], frame.shape[0], 1])
            kps = kps_normalized_xyc * scale_array
            
        
        conf_ok = False
        skeleton_colors = {}
        for line in SKELETON_LINES: skeleton_colors[line] = default_color
            
        if kps is not None:
            # Note: We still check critical joints, but the KP_CONF_MIN is lower now
            critical_joints = [KS['ls'], KS['rs'], KS['le'], KS['re'], KS['lh'], KS['rh']]
            conf_ok = all(kps[j, 2] > KP_CONF_MIN for j in critical_joints)
        
        if kps is None or not conf_ok:
            cv2.putText(frame, "ADJUST CAMERA: Low Confidence/No Detection", (20, 150), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 165, 255), 2)
            prev_y = None
            
        else:
            # --- Rep Counting Normalization ---
            shoulder_y = (kps[KS['ls'], 1] + kps[KS['rs'], 1]) / 2
            hip_y = (kps[KS['lh'], 1] + kps[KS['rh'], 1]) / 2
            
            shoulder_hip_distance = abs(shoulder_y - hip_y)
            # The movement threshold is based on the shoulder-hip distance, which scales with distance
            dynamic_move_threshold = shoulder_hip_distance * RELATIVE_MOVE_THRESHOLD 
            effective_move_threshold = max(5, dynamic_move_threshold) # Reduced minimum from 10 to 5

            # --- State Tracking ---
            if prev_y is not None:
                delta_y = shoulder_y - prev_y
                
                # UP -> DOWN transition (Moving down = shoulder y increases)
                if state == "up" and delta_y > effective_move_threshold:
                    state = "down"
                    min_elbow_angle = 180.0
                    
                # DOWN -> UP transition (Rep Completion: Moving up = shoulder y decreases)
                elif state == "down" and delta_y < -effective_move_threshold:
                    state = "up"
                    
                    # --- FINAL VERDICT EVALUATION ---
                    reps += 1
                    final_feedback = "CORRECT FORM!"
                    feedback_color = (0, 255, 0) 
                    geo_issues = []
                    
                    current_min_angle = get_best_elbow_angle(kps)
                    current_hip_angle = get_hip_angle(kps)
                    
                    # 1. Elbow Depth Check
                    if min_elbow_angle > ELBOW_ANGLE_THRESHOLD:
                        geo_issues.append(f"Go deeper! (Min angle: {int(min_elbow_angle)}°)")
                    
                    # 2. Full Extension Check
                    if current_min_angle < EXTENDED_ELBOW_THRESHOLD:
                        geo_issues.append("Extend arms fully (No Lockout)")
                        
                    # 3. AI Classifier Check (for hip/back form)
                    features = create_robust_features(kps) 
                    features_array = np.array(features).reshape(1, -1)
                    
                    ai_verdict_idx = rf_model.predict(features_array)[0]
                    ai_proba = rf_model.predict_proba(features_array)[0]
                    
                    if ai_verdict_idx == 'correct':
                        confidence = ai_proba[rf_model.classes_ == 'correct'][0]
                    else:
                        confidence = ai_proba[rf_model.classes_ == 'incorrect'][0]

                    ai_feedback = f" (AI: {ai_verdict_idx}, conf: {confidence:.2f})"

                    if ai_verdict_idx == 'incorrect' and confidence >= AI_CONF_MIN:
                        if current_hip_angle < HIP_ANGLE_THRESHOLD:
                             geo_issues.append(f"Keep back straight! (Hip angle: {int(current_hip_angle)}°)")
                        else:
                             geo_issues.append("Subtle Form Breakdown (AI flag)")
                            
                    # Final Decision Logic
                    if geo_issues:
                        final_feedback = ", ".join(geo_issues)
                        feedback_color = (0, 0, 255) # Red for any failure
                        
                        # Color the skeleton red if any issue is flagged
                        for seg in SKELETON_LINES: skeleton_colors[seg] = (0, 0, 255) 
                        
                        arm_segments = [(KS['ls'], KS['le']), (KS['le'], KS['lw']), (KS['rs'], KS['re']), (KS['re'], KS['rw'])]
                        hip_segments = [(KS['ls'], KS['lh']), (KS['rs'], KS['rh']), (KS['lh'], KS['rh'])]
                        
                        if any(s in geo_issues for s in ["Go deeper!", "Extend arms fully (No Lockout)"]):
                            for seg in arm_segments: skeleton_colors[seg] = (0, 0, 255)
                        
                        if any(s in geo_issues for s in ["Keep back straight!", "Subtle Form Breakdown (AI flag)"]):
                            for seg in hip_segments: skeleton_colors[seg] = (0, 0, 255)
                            
                    else:
                        for line in SKELETON_LINES: skeleton_colors[line] = (0, 255, 0) # Green for perfect form

                    print(f"Rep {reps}: {final_feedback}{ai_feedback}")
                    feedback_buffer = [final_feedback, feedback_color, time.time() + 3]
                
            # --- DYNAMIC FEEDBACK (DURING 'down' state) ---
            if state == "down":
                current_elbow_angle = get_best_elbow_angle(kps)
                if current_elbow_angle < min_elbow_angle: min_elbow_angle = current_elbow_angle

                # Dynamic Depth Warning (Orange for arms)
                if current_elbow_angle > ELBOW_ANGLE_THRESHOLD + 10: 
                    feedback_buffer = ["GO DEEPER!", (0, 165, 255), time.time() + 0.5]
                    arm_segments = [(KS['ls'], KS['le']), (KS['le'], KS['lw']), (KS['rs'], KS['re']), (KS['re'], KS['rw'])]
                    for seg in arm_segments: skeleton_colors[seg] = (0, 165, 255)

                # Dynamic Hip Warning (Red for torso/hips)
                current_hip_angle = get_hip_angle(kps)
                if current_hip_angle < HIP_ANGLE_THRESHOLD - 5:
                    feedback_buffer = ["WARNING: FIX HIPS/BACK NOW!", (0, 0, 255), time.time() + 0.5]
                    hip_segments = [(KS['ls'], KS['lh']), (KS['rs'], KS['rh']), (KS['lh'], KS['rh'])]
                    for seg in hip_segments: skeleton_colors[seg] = (0, 0, 255)
                    
            prev_y = shoulder_y


        # --- 4. DRAW SKELETON (UI) ---
        if kps is not None and conf_ok:
            for p1_idx, p2_idx in SKELETON_LINES:
                p1, p2 = kps[p1_idx], kps[p2_idx]
                segment_color = skeleton_colors.get((p1_idx, p2_idx), default_color)

                if p1[2] > KP_CONF_MIN and p2[2] > KP_CONF_MIN: 
                    cv2.line(frame, (int(p1[0]), int(p1[1])), (int(p2[0]), int(p2[1])), 
                             segment_color, 2)
            
            for i in range(kps.shape[0]):
                kp = kps[i]
                if kp[2] > KP_CONF_MIN:
                    cv2.circle(frame, (int(kp[0]), int(kp[1])), 5, default_color, -1) 

        # --- 5. Draw Text UI ---
        cv2.putText(frame, f"Reps: {reps}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
        
        cv2.putText(frame, f"State: {state.upper()}", (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        if feedback_buffer and time.time() < feedback_buffer[2]:
            text, color, _ = feedback_buffer
            cv2.putText(frame, text, (20, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Pushup Trainer", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): break

    # cleanup
    cap.release()
    cv2.destroyAllWindows()

else:
    print("Cannot run live loop due to model loading error.")

Models loaded successfully. Using Robust Feature Engineering.
Rep 1: Extend arms fully (No Lockout), Keep back straight! (Hip angle: 50°) (AI: incorrect, conf: 0.73)
Rep 2: Go deeper! (Min angle: 156°), Extend arms fully (No Lockout), Keep back straight! (Hip angle: 99°) (AI: incorrect, conf: 0.91)
Rep 3: Go deeper! (Min angle: 125°), Extend arms fully (No Lockout), Keep back straight! (Hip angle: 115°) (AI: incorrect, conf: 0.65)
Rep 4: Extend arms fully (No Lockout), Keep back straight! (Hip angle: 125°) (AI: incorrect, conf: 0.57)
Rep 5: Extend arms fully (No Lockout), Keep back straight! (Hip angle: 135°) (AI: incorrect, conf: 0.63)
Rep 6: Go deeper! (Min angle: 152°), Extend arms fully (No Lockout) (AI: correct, conf: 0.53)
Rep 7: Extend arms fully (No Lockout), Keep back straight! (Hip angle: 138°) (AI: incorrect, conf: 0.63)
Rep 8: Extend arms fully (No Lockout), Keep back straight! (Hip angle: 131°) (AI: incorrect, conf: 0.57)
Rep 9: Extend arms fully (No Lockout), Keep back st